In [1]:
import typing
type_int = typing.Union[int,Integer]

# Class of Feistel Ciphers
class Feistel:
    # This function should accept a key in bytearray.
    # This function should preprocess the key, i.e. preparing the key schedule
    def __init__(self, key: bytearray): pass

    # This function is the function f.
    # |    L    |   |    R    |
    #      |             |
    #     (+)--<-(f)--<--|
    #      |             |
    # |    R    |   |    L    |   (!) Swapped
    # D: The data, either L or R, to be processed
    # n: count of iterations. the first iteration have n=1. 
    #    (Not part of Feistel but nesserary to DES etc)
    def f_func(self, D: bytearray, n: type_int) -> bytearray:
        # Dummy function to not modify anything. THIS IS NOT ENCRYPTION!
        return D

    # This function prepares the initial permutation (IP).
    # RTN: IP
    def prepare_ip(self, DATA: bytearray) -> bytearray:
        # Dummy function to not modify anything. THIS IS NOT ENCRYPTION!
        return DATA

    # This function defines an single encrypt interation.
    # L: The upper half of the data. R: the rest
    # n: count of iterations. the first iteration have n=1.
    def encrypt_iteration(self, L: bytearray, R: bytearray, n: type_int) -> typing.Tuple[bytearray, bytearray]: 
        f_rtn = self.f_func(R, n)
        processed = bytearray([L[i] ^^ f_rtn[i] for i in range(len(L))])
        # L, R
        return R, processed

    # This function defines an single decrypt interation.
    # L: The upper half of the data. R: the rest
    # n: count of iterations (in their order of encryption). the first iteration have n=1.
    def decrypt_iteration(self, L: bytearray, R: bytearray, n: type_int) -> typing.Tuple[bytearray, bytearray]: 
        f_rtn = self.f_func(L, n)
        processed = bytearray([R[i] ^^ f_rtn[i] for i in range(len(L))])
        # L, R
        return processed, L

    # Number of iterations to be done.
    # This should be overriden by class definitions.
    num_iterations = 16
    
    # This function is the encryption function.
    # DATA: The data block to be encrypted.
    # RTN: The ciphertext
    def encrypt(self, DATA: bytearray):
        len_data = len(DATA)
        assert len_data % 2 == 0
        half_len = int(len_data / 2)
        DATA_IP = self.prepare_ip(DATA)
        L,R = DATA[:half_len], DATA[half_len:]
        for i in range(self.num_iterations):
            L,R = self.encrypt_iteration(L,R,i+1)
        return L + R

    # This function is the decryption function.
    # DATA: The data block to be decrypted.
    # RTN: The plain text
    def decrypt(self, DATA: bytearray):
        len_data = len(DATA)
        assert len_data % 2 == 0
        half_len = int(len_data / 2)
        DATA_IP = self.prepare_ip(DATA)
        L,R = DATA[:half_len], DATA[half_len:]
        for i in range(self.num_iterations):
            L,R = self.decrypt_iteration(L,R,self.num_iterations - i)
        return L + R

In [2]:
# Convert bytearray to sth like FF FA 01 BB
def bytearray_to_readable(btarr):
    return " ".join([hex(b)[2:].upper().zfill(2) for b in btarr])

In [3]:
fei = Feistel(bytearray())

m = "We are discovered, safe yourself.."
m_btarr = bytearray(m,encoding="ascii")
c = fei.encrypt(m_btarr)
rtn_m_btarr = fei.decrypt(c)
rtn_m = rtn_m_btarr.decode(encoding="ascii")

print(m,bytearray_to_readable(m_btarr))
print(bytearray_to_readable(c))
print(rtn_m,bytearray_to_readable(rtn_m_btarr))

We are discovered, safe yourself.. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E 2E
2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E 2E 7B 45 53 00 14 00 00 1D 06 06 11 1C 13 09 14 4B 4A
We are discovered, safe yourself.. 57 65 20 61 72 65 20 64 69 73 63 6F 76 65 72 65 64 2C 20 73 61 66 65 20 79 6F 75 72 73 65 6C 66 2E 2E
